# SECTION 1: INSTALLATION AND SETUP

In [1]:
# MediaPipe Body Tracking - Educational Tutorial
# A step-by-step guide to full-body tracking, masking, and privacy protection

import sys
import os

# Check if we're in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("🔵 Running in Google Colab")
except ImportError: # assuming you have conda installed
    IN_COLAB = False
    print("🟢 Google Colab was not detected.\n" \
           "Create a virtual environment and select that as your kernel.\n" \
             "An example is shown in comments below.") 
    print("You may be asked to install ipykernel in your virtual environment.")
    # conda create -n tt_masking -y python=3.11.13
    # conda activate tt_masking   

🟢 Google Colab was not detected.
Create a virtual environment and select that as your kernel.
An example is shown in comments below.
You may be asked to install ipykernel in your virtual environment.


In [2]:
print("\n📦 STEP 1: Installing required packages...")
!pip install mediapipe opencv-python-headless matplotlib pandas ipywidgets tqdm


📦 STEP 1: Installing required packages...


In [55]:
# Import required libraries
import io
import base64
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from tqdm.auto import tqdm
from IPython.display import display, Video, HTML, clear_output

print("✅ All packages imported successfully!")

✅ All packages imported successfully!


# SECTION 2: UNDERSTANDING MEDIAPIPE ANATOMY

In [4]:
class MediaPipeAnatomy:
    """
    A class to help students understand MediaPipe's tracking capabilities
    """

    def __init__(self):
        # Body landmarks (33 points)
        self.body_landmarks = [
            'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER',
            'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
            'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT',
            'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW',
            'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY',
            'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB',
            'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE',
            'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL',
            'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
        ]

        # Hand landmarks (21 points per hand)
        hand_parts = ['WRIST', 'THUMB_CMC', 'THUMB_MCP', 'THUMB_IP', 'THUMB_TIP',
                     'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP', 'INDEX_FINGER_DIP', 'INDEX_FINGER_TIP',
                     'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_DIP', 'MIDDLE_FINGER_TIP',
                     'RING_FINGER_MCP', 'RING_FINGER_PIP', 'RING_FINGER_DIP', 'RING_FINGER_TIP',
                     'PINKY_FINGER_MCP', 'PINKY_FINGER_PIP', 'PINKY_FINGER_DIP', 'PINKY_FINGER_TIP']

        self.hand_landmarks = []
        for side in ['LEFT', 'RIGHT']:
            for part in hand_parts:
                self.hand_landmarks.append(f"{side}_{part}") # ex: wrist is added as left_wrist and right_wrist

        # Face landmarks (478 points)
        self.face_landmarks = [str(i) for i in range(478)]

    def show_summary(self):
        """Display a summary of tracking capabilities"""
        print("🎯 MediaPipe Tracking Capabilities:")
        print(f"   👤 Body: {len(self.body_landmarks)} keypoints")
        print(f"   ✋ Hands: {len(self.hand_landmarks)} keypoints (21 per hand)")
        print(f"   😊 Face: {len(self.face_landmarks)} keypoints")
        print(f"   📊 Total: {len(self.body_landmarks) + len(self.hand_landmarks) + len(self.face_landmarks)} tracking points!")

        return {
            'body': len(self.body_landmarks),
            'hands': len(self.hand_landmarks),
            'face': len(self.face_landmarks)
        }

    def create_column_names(self):
        """Create column names for CSV export"""
        body_columns = ['time']
        hand_columns = ['time']
        face_columns = ['time']

        # Body columns (X, Y, Z, visibility)
        for landmark in self.body_landmarks:
            for coord in ['X', 'Y', 'Z', 'visibility']:
                body_columns.append(f"{coord}_{landmark}")

        # Hand columns (X, Y, Z)
        for landmark in self.hand_landmarks:
            for coord in ['X', 'Y', 'Z']:
                hand_columns.append(f"{coord}_{landmark}")

        # Face columns (X, Y, Z)
        for landmark in self.face_landmarks:
            for coord in ['X', 'Y', 'Z']:
                face_columns.append(f"{coord}_{landmark}")

        return body_columns, hand_columns, face_columns

In [5]:
print("\n🧠 STEP 2: Understanding what MediaPipe can track...")
anatomy = MediaPipeAnatomy()
tracking_info = anatomy.show_summary()
body_cols, hand_cols, face_cols = anatomy.create_column_names()


🧠 STEP 2: Understanding what MediaPipe can track...
🎯 MediaPipe Tracking Capabilities:
   👤 Body: 33 keypoints
   ✋ Hands: 42 keypoints (21 per hand)
   😊 Face: 478 keypoints
   📊 Total: 553 tracking points!


# SECTION 3: CONFIGURATION


In [33]:
class MediaPipeConfig:
    """
    Interactive configuration for MediaPipe processing
    """

    def __init__(self):
        self.setup_widgets()

    def setup_widgets(self):
        """Create interactive widgets for configuration"""

        # Privacy options
        self.skeleton_widget = widgets.Checkbox(value=True, description='Show skeleton overlay')
        self.face_only_widget = widgets.Checkbox(value=False, description='Face skeleton only')
        self.white_bg_widget = widgets.Checkbox(value=False, description='White background mode')

        # Masking options
        self.mask_body_widget = widgets.Checkbox(value=False, description='Mask body (black silhouette)')
        self.mask_face_widget = widgets.Checkbox(value=False, description='Mask face (black)')

        # Blurring options
        self.blur_body_widget = widgets.Checkbox(value=True, description='Blur body')
        self.blur_face_widget = widgets.Checkbox(value=False, description='Blur face')
        self.blur_intensity_widget = widgets.FloatSlider(
            value=1.0, min=0.0, max=1.0, step=0.1,
            description='Blur intensity:', style={'description_width': 'initial'}
        )

        # Trace options
        self.finger_traces_widget = widgets.Checkbox(value=True, description='Add finger traces')
        self.trace_length_widget = widgets.FloatSlider(
            value=2.0, min=0.5, max=5.0, step=0.5,
            description='Trace length (seconds):', style={'description_width': 'initial'}
        )

    def show_interface(self):
        """Display the configuration interface"""

        privacy_box = widgets.VBox([
            widgets.HTML("<h3>🔒 Privacy & Visualization</h3>"),
            self.skeleton_widget,
            self.face_only_widget,
            self.white_bg_widget
        ])

        masking_box = widgets.VBox([
            widgets.HTML("<h3>🎭 Masking Options</h3>"),
            self.mask_body_widget,
            self.mask_face_widget
        ])

        blurring_box = widgets.VBox([
            widgets.HTML("<h3>🌫️ Blurring Options</h3>"),
            self.blur_body_widget,
            self.blur_face_widget,
            self.blur_intensity_widget
        ])

        tracing_box = widgets.VBox([
            widgets.HTML("<h3>✏️ Movement Tracing</h3>"),
            self.finger_traces_widget,
            self.trace_length_widget
        ])

        config_interface = widgets.HBox([privacy_box, masking_box, blurring_box, tracing_box])

        return config_interface

    def get_config(self):
        """Get current configuration as dictionary"""
        return {
            'skeleton': self.skeleton_widget.value,
            'skeleton_face_only': self.face_only_widget.value,
            'whitebackground': self.white_bg_widget.value,
            'maskingbody': self.mask_body_widget.value,
            'maskingface': self.mask_face_widget.value,
            'blurringbody': self.blur_body_widget.value,
            'blurringface': self.blur_face_widget.value,
            'blurringfactor': self.blur_intensity_widget.value,
            'add_finger_traces': self.finger_traces_widget.value,
            'trace_length_seconds': self.trace_length_widget.value,
            'trace_color_left': (0, 255, 0),  # Green
            'trace_color_right': (0, 0, 255)  # Blue
        }

In [34]:
print("\n⚙️ STEP 3: Interactive configuration panel...")
config_panel = MediaPipeConfig()


⚙️ STEP 3: Interactive configuration panel...


# SECTION 4: HELPER FUNCTIONS


In [31]:
def analyze_csv_output(video_name):
    """
    Analyze and visualize the CSV output data

    Args:
        video_name (str): Name of the processed video (without extension)
    """
    print(f"📊 Analyzing CSV data for: {video_name}")

    try:
        # Load CSV files
        body_df = pd.read_csv(f'Output_TimeSeries/{video_name}_body.csv')
        hand_df = pd.read_csv(f'Output_TimeSeries/{video_name}_hands.csv')
        face_df = pd.read_csv(f'Output_TimeSeries/{video_name}_face.csv')

        print(f"✅ Loaded data files:")
        print(f"   📊 Body: {len(body_df)} frames, {len(body_df.columns)} columns")
        print(f"   ✋ Hand: {len(hand_df)} frames, {len(hand_df.columns)} columns")
        print(f"   😊 Face: {len(face_df)} frames, {len(face_df.columns)} columns")

        # Create visualizations
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle(f'MediaPipe Analysis: {video_name}', fontsize=16)

        # Plot 1: Body landmark visibility over time
        axes[0, 0].plot(body_df['time'], body_df['visibility_NOSE'], label='Nose', alpha=0.7)
        axes[0, 0].plot(body_df['time'], body_df['visibility_LEFT_WRIST'], label='Left Wrist', alpha=0.7)
        axes[0, 0].plot(body_df['time'], body_df['visibility_RIGHT_WRIST'], label='Right Wrist', alpha=0.7)
        axes[0, 0].set_title('Body Landmark Visibility')
        axes[0, 0].set_xlabel('Time (ms)')
        axes[0, 0].set_ylabel('Visibility Score')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

        # Plot 2: Hand movement (if available)
        if 'X_LEFT_INDEX_FINGER_TIP' in hand_df.columns:
            # Calculate hand velocities
            left_x = hand_df['X_LEFT_INDEX_FINGER_TIP'].dropna()
            left_y = hand_df['Y_LEFT_INDEX_FINGER_TIP'].dropna()

            if len(left_x) > 1:
                left_velocity = np.sqrt(np.diff(left_x)**2 + np.diff(left_y)**2)
                axes[0, 1].plot(left_velocity, label='Left Index Finger', alpha=0.7)
                axes[0, 1].set_title('Hand Movement Velocity')
                axes[0, 1].set_xlabel('Frame')
                axes[0, 1].set_ylabel('Velocity (normalized units)')
                axes[0, 1].legend()
                axes[0, 1].grid(True, alpha=0.3)

        # Plot 3: Body pose trajectory (nose movement)
        axes[1, 0].plot(body_df['X_NOSE'], body_df['Y_NOSE'], 'o-', alpha=0.6, markersize=2)
        axes[1, 0].set_title('Head Movement Trajectory')
        axes[1, 0].set_xlabel('X Position (normalized)')
        axes[1, 0].set_ylabel('Y Position (normalized)')
        axes[1, 0].invert_yaxis()  # Invert Y axis to match image coordinates
        axes[1, 0].grid(True, alpha=0.3)

        # Plot 4: Data completeness heatmap
        completeness_data = []
        for col in ['Body', 'Hands', 'Face']:
            if col == 'Body':
                df_subset = body_df.select_dtypes(include=[np.number])
            elif col == 'Hands':
                df_subset = hand_df.select_dtypes(include=[np.number])
            else:
                df_subset = face_df.select_dtypes(include=[np.number])

            completeness = (1 - df_subset.isnull().sum() / len(df_subset)) * 100
            completeness_data.append(completeness.mean())

        axes[1, 1].bar(['Body', 'Hands', 'Face'], completeness_data,
                      color=['skyblue', 'lightgreen', 'lightcoral'], alpha=0.7)
        axes[1, 1].set_title('Data Completeness')
        axes[1, 1].set_ylabel('Completeness (%)')
        axes[1, 1].set_ylim(0, 100)
        axes[1, 1].grid(True, alpha=0.3)

        # Add percentage labels on bars
        for i, v in enumerate(completeness_data):
            axes[1, 1].text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom')

        plt.tight_layout()
        plt.show()

        # Print summary statistics
        print(f"\n📈 Summary Statistics:")
        print(f"   ⏱️ Duration: {body_df['time'].max()/1000:.1f} seconds")
        print(f"   📊 Data completeness:")
        print(f"      👤 Body: {completeness_data[0]:.1f}%")
        print(f"      ✋ Hands: {completeness_data[1]:.1f}%")
        print(f"      😊 Face: {completeness_data[2]:.1f}%")

        return {
            'body_df': body_df,
            'hand_df': hand_df,
            'face_df': face_df,
            'completeness': completeness_data
        }

    except FileNotFoundError as e:
        print(f"❌ Could not find CSV files for {video_name}")
        print(f"Error: {e}")
        return None

def upload_sample_video():
    """
    Helper function to upload sample video in Colab
    """
    if IN_COLAB:
        from google.colab import files
        print("📤 Upload a video file for testing:")
        uploaded = files.upload()

        for filename in uploaded.keys():
            # Move uploaded file to Input_Videos folder
            import shutil
            shutil.move(filename, f'Input_Videos/{filename}')
            print(f"✅ Uploaded {filename} to Input_Videos/")

            # Show video info
            show_video_info(f'Input_Videos/{filename}')

        return list(uploaded.keys())
    else:
        print("📁 Please manually add video files to the Input_Videos/ folder")
        return []

def get_video_list(video_dir: str):
    if not os.path.exists(video_dir):
        raise FileNotFoundError(f"Video directory {video_dir} does not exist")
    
    videos = [f for f in os.listdir(video_dir)
               if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    return videos

def parse_mediapipe_landmarks(landmarks):
    """
    Convert MediaPipe landmarks to clean numerical values

    Args:
        landmarks: MediaPipe landmark object

    Returns:
        list: Clean numerical coordinates
    """
    if landmarks is None:
        return []

    coordinates = []
    for landmark in landmarks.landmark:
        coordinates.extend([str(landmark.x), str(landmark.y), str(landmark.z)])

    return coordinates

def create_folders():
    """Create necessary folders for input/output"""
    folders = ['Input_Videos', 'Output_Videos', 'Output_TimeSeries']

    for folder in folders:
        if not os.path.exists(folder):
            os.makedirs(folder)
            print(f"📁 Created folder: {folder}")
        else:
            print(f"✅ Folder exists: {folder}")

    return folders

def get_video_specs(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise f"❌ Could not open video: {video_path}"
        
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps if fps > 0 else 0

    cap.release()

    return fps, frame_count, width, height, duration

def show_video_info(video_path):
    """Display information about the input video"""

    fps, frame_count, width, height, duration = get_video_specs(video_path)

    info = {
        'fps': fps,
        'frame_count': frame_count,
        'width': width,
        'height': height,
        'duration': duration
    }

    print(f"📹 Video Information:")
    print(f"   📏 Dimensions: {width} x {height}")
    print(f"   🎬 Frame rate: {fps:.1f} FPS")
    print(f"   ⏱️ Duration: {duration:.1f} seconds")
    print(f"   🎞️ Total frames: {frame_count}")

    return info

def display_frame_with_landmarks(image, results, config):
    """
    Display a single frame with MediaPipe landmarks for demonstration

    Args:
        image: Input image
        results: MediaPipe results
        config: Configuration dictionary
    """
    # Initialize MediaPipe drawing utilities
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_holistic = mp.solutions.holistic

    # Convert image for display
    display_image = image.copy()

    if results.pose_landmarks:
        if config['skeleton']:
            if not config['skeleton_face_only']:
                # Draw pose landmarks
                mp_drawing.draw_landmarks(
                display_image,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                )
                # Draw hand landmarks
                mp_drawing.draw_landmarks(
                display_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                mp_drawing.draw_landmarks(
                display_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

            # Draw face landmarks
            mp_drawing.draw_landmarks(
                display_image,
                results.face_landmarks,
                mp_holistic.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
            )

    return display_image

In [32]:
print("\n🔧 STEP 4: Creating Folders")
created_folders = create_folders()


🔧 STEP 4: Creating Folders
✅ Folder exists: Input_Videos
✅ Folder exists: Output_Videos
✅ Folder exists: Output_TimeSeries


# SECTION 5: MAIN PROCESSING CLASS


In [30]:
print("\n🚀 STEP 5: Understand the Main processing engine...")

class MediaPipeProcessor:
    """
    Main class for processing videos with MediaPipe
    """
    def __init__(self, config):
        self.config = config
        self.mp_holistic = mp.solutions.holistic
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles

    def process_single_video(self, video_path, output_video_path=None, save_csv=True):
        """
        Process a single video with MediaPipe

        Args:
            video_path (str): Path to input video
            output_video_path (str): Path for output video (optional)
            save_csv (bool): Whether to save CSV data

        Returns:
            dict: Processing results and data
        """
        cap = cv2.VideoCapture(video_path)
        fps, total_frames, width, height, _ = get_video_specs(video_path)

        # Setup video writer if output path provided
        if output_video_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

        # Initialize data storage
        body_data = [body_cols]
        hand_data = [hand_cols]
        face_data = [face_cols]

        # Initialize finger trace buffers
        if self.config['add_finger_traces']:
            trace_frames = int(self.config['trace_length_seconds'] * fps)
            left_finger_trace = []
            right_finger_trace = []

        # Process video frame by frame
        frame_count = 0
        time_ms = 0

        with self.mp_holistic.Holistic(
            static_image_mode=False,
            enable_segmentation=True,
            refine_face_landmarks=True
        ) as holistic:

            # Progress bar for long videos
            pbar = tqdm(total=total_frames, desc="Processing frames")

            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                # Convert BGR to RGB for MediaPipe
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Process frame with MediaPipe
                results = holistic.process(rgb_frame)

                # Create output frame
                output_frame = self._apply_effects(frame, results)

                # Add finger traces if enabled
                if self.config['add_finger_traces']:
                    output_frame = self._add_finger_traces(
                        output_frame, results,
                        left_finger_trace, right_finger_trace,
                        trace_frames, width, height
                    )

                # Draw skeleton if enabled
                if self.config['skeleton']:
                    output_frame = self._draw_skeleton(output_frame, results)

                # Save frame to output video
                if output_video_path:
                    out.write(output_frame)

                # Extract and save landmark data
                self._extract_landmark_data(
                    results, time_ms, body_data, hand_data, face_data
                )

                # Update progress
                frame_count += 1
                time_ms += (1000 / fps)
                pbar.update(1)

                # Show progress every 30 frames
                if frame_count % 30 == 0:
                    pbar.set_postfix({
                        'Time': f"{time_ms/1000:.1f}s",
                        'Pose': '✅' if results.pose_landmarks else '❌'
                    })

        # Cleanup
        pbar.close()
        cap.release()
        if output_video_path:
            out.release()

        # Save CSV data if requested
        if save_csv:
            video_name = os.path.splitext(os.path.basename(video_path))[0]
            self._save_csv_data(video_name, body_data, hand_data, face_data)

        return {
            'frames_processed': frame_count,
            'duration': time_ms / 1000,
            'body_data': body_data,
            'hand_data': hand_data,
            'face_data': face_data
        }

    def _apply_effects(self, frame, results):
        """Apply masking and blurring effects"""
        h, w, c = frame.shape
        output_frame = frame.copy()

        # White background mode
        if self.config['whitebackground']:
            output_frame = np.full((h, w, 3), 255, dtype=np.uint8)
            return output_frame

        # Body blurring
        if self.config['blurringbody'] and results.segmentation_mask is not None:
            # Create body mask
            mask = (results.segmentation_mask > 0.5).astype(np.uint8)

            # Apply Gaussian blur
            kernel_size = int(51 * self.config['blurringfactor'])
            if kernel_size % 2 == 0:
                kernel_size += 1

            blurred = cv2.GaussianBlur(output_frame, (kernel_size, kernel_size), 0)

            # Blend blurred and original
            mask_3d = np.stack([mask] * 3, axis=-1)
            output_frame = np.where(mask_3d, blurred, output_frame)

        # Body masking
        if self.config['maskingbody'] and results.segmentation_mask is not None:
            mask = (results.segmentation_mask > 0.5).astype(np.uint8)
            mask_3d = np.stack([mask] * 3, axis=-1)
            output_frame = np.where(mask_3d, 0, output_frame)

        # Face effects
        if results.face_landmarks and (self.config['blurringface'] or self.config['maskingface']):
            # Create face mask
            face_points = []
            for landmark in results.face_landmarks.landmark:
                x = int(landmark.x * w)
                y = int(landmark.y * h)
                face_points.append([x, y])

            face_points = np.array(face_points, dtype=np.int32)
            face_mask = np.zeros((h, w), dtype=np.uint8)
            cv2.fillConvexPoly(face_mask, cv2.convexHull(face_points), 255)

            if self.config['blurringface']:
                kernel_size = int(51 * self.config['blurringfactor'])
                if kernel_size % 2 == 0:
                    kernel_size += 1
                blurred = cv2.GaussianBlur(output_frame, (kernel_size, kernel_size), 0)
                face_mask_3d = np.stack([face_mask] * 3, axis=-1) / 255.0
                output_frame = (output_frame * (1 - face_mask_3d) + blurred * face_mask_3d).astype(np.uint8)

            if self.config['maskingface']:
                output_frame[face_mask > 0] = 0

        return output_frame

    def _add_finger_traces(self, frame, results, left_trace, right_trace, max_frames, width, height):
        """Add finger movement traces"""
        # Get current finger positions
        if results.left_hand_landmarks:
            landmark = results.left_hand_landmarks.landmark[8]  # Index finger tip
            pos = (int(landmark.x * width), int(landmark.y * height))
            left_trace.append(pos)

        if results.right_hand_landmarks:
            landmark = results.right_hand_landmarks.landmark[8]  # Index finger tip
            pos = (int(landmark.x * width), int(landmark.y * height))
            right_trace.append(pos)

        # Limit trace length
        if len(left_trace) > max_frames:
            left_trace.pop(0)
        if len(right_trace) > max_frames:
            right_trace.pop(0)

        # Draw traces
        for i, trace in enumerate([left_trace, right_trace]):
            color = self.config['trace_color_left'] if i == 0 else self.config['trace_color_right']

            for j in range(len(trace) - 1):
                if j + 1 < len(trace):
                    alpha = (j + 1) / len(trace)
                    overlay = frame.copy()
                    cv2.line(overlay, trace[j], trace[j + 1], color, 2)
                    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

        return frame

    def _draw_skeleton(self, frame, results):
        """Draw skeleton landmarks"""
        if self.config['skeleton_face_only']:
            # Draw only face
            if results.face_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame, results.face_landmarks,
                    self.mp_holistic.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=self.mp_drawing_styles.get_default_face_mesh_tesselation_style()
                )
        else:
            # Draw full skeleton
            if results.pose_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame, results.pose_landmarks,
                    self.mp_holistic.POSE_CONNECTIONS,
                    landmark_drawing_spec=self.mp_drawing_styles.get_default_pose_landmarks_style()
                )

            if results.left_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame, results.left_hand_landmarks,
                    self.mp_holistic.HAND_CONNECTIONS
                )

            if results.right_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame, results.right_hand_landmarks,
                    self.mp_holistic.HAND_CONNECTIONS
                )

            if results.face_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame, results.face_landmarks,
                    self.mp_holistic.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=self.mp_drawing_styles.get_default_face_mesh_tesselation_style()
                )

        return frame

    def _extract_landmark_data(self, results, time_ms, body_data, hand_data, face_data):
        """Extract landmark coordinates for CSV export"""
        # Body data
        if results.pose_landmarks:
            body_row = [time_ms]
            for landmark in results.pose_landmarks.landmark:
                body_row.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
        else:
            body_row = [time_ms] + [np.nan] * (len(body_cols) - 1)
        body_data.append(body_row)

        # Hand data
        hand_row = [time_ms]

        # Left hand
        if results.left_hand_landmarks:
            for landmark in results.left_hand_landmarks.landmark:
                hand_row.extend([landmark.x, landmark.y, landmark.z])
        else:
            hand_row.extend([np.nan] * 63)  # 21 landmarks * 3 coordinates

        # Right hand
        if results.right_hand_landmarks:
            for landmark in results.right_hand_landmarks.landmark:
                hand_row.extend([landmark.x, landmark.y, landmark.z])
        else:
            hand_row.extend([np.nan] * 63)

        hand_data.append(hand_row)

        # Face data
        if results.face_landmarks:
            face_row = [time_ms]
            for landmark in results.face_landmarks.landmark:
                face_row.extend([landmark.x, landmark.y, landmark.z])
        else:
            face_row = [time_ms] + [np.nan] * (len(face_cols) - 1)
        face_data.append(face_row)

    def _save_csv_data(self, video_name, body_data, hand_data, face_data):
        """Save extracted data to CSV files"""
        # Save body data
        body_df = pd.DataFrame(body_data[1:], columns=body_data[0])
        body_df.to_csv(f'Output_TimeSeries/{video_name}_body.csv', index=False)

        # Save hand data
        hand_df = pd.DataFrame(hand_data[1:], columns=hand_data[0])
        hand_df.to_csv(f'Output_TimeSeries/{video_name}_hands.csv', index=False)

        # Save face data
        face_df = pd.DataFrame(face_data[1:], columns=face_data[0])
        face_df.to_csv(f'Output_TimeSeries/{video_name}_face.csv', index=False)

        print(f"💾 Saved CSV files for {video_name}")
        print(f"   📊 Body data: {len(body_df)} frames, {len(body_df.columns)} columns")
        print(f"   ✋ Hand data: {len(hand_df)} frames, {len(hand_df.columns)} columns")
        print(f"   😊 Face data: {len(face_df)} frames, {len(face_df.columns)} columns")


🚀 STEP 5: Understand the Main processing engine...


# SECTION 6: DEMONSTRATION AND TESTING


In [22]:
def demo_processing():
    """
    Demonstration function that can be called after setup
    """
    print("🎯 Demo: Processing videos with current configuration...")

    # Get current configuration
    current_config = config_panel.get_config()

    # Show configuration summary
    print("\n📋 Current Configuration:")
    for key, value in current_config.items():
        if isinstance(value, bool):
            status = "✅" if value else "❌"
            print(f"   {status} {key}: {value}")
        else:
            print(f"   🔧 {key}: {value}")

    # Check for videos in input folder
    input_videos = get_video_list('Input_Videos')

    if not input_videos:
        print("\n⚠️ No videos found in Input_Videos folder!")
        print("Please add a video file to Input_Videos/ folder and try again.")
        return

    print(f"\n📹 Found {len(input_videos)} video(s) to process:")
    for video in input_videos:
        print(f"   🎬 {video}")

    # Process each video
    processor = MediaPipeProcessor(current_config)

    for video in input_videos:
        print(f"\n🚀 Processing: {video}")

        input_path = f"Input_Videos/{video}"
        output_path = f"Output_Videos/{video}"

        # Show video info
        video_info = show_video_info(input_path)

        if video_info is None:
            continue
        try:
            # Process the video
            results = processor.process_single_video(
                input_path, output_path, save_csv=True
            )

            print(f"✅ Successfully processed {results['frames_processed']} frames")
            print(f"⏱️ Video duration: {results['duration']:.1f} seconds")
            print(f"📁 Output saved to: {output_path}")

        except Exception as e:
            print(f"❌ Error processing {video}: {str(e)}")

    print("\n🎉 Processing complete! Check your Output_Videos and Output_TimeSeries folders.")

In [23]:
print("✅ All setup complete! Ready to process videos.")
print("1. Configure your processing options below:")
print("2. Add video files to the 'Input_Videos' folder")
print("3. Run demo_processing() to start processing")
config_panel.show_interface()

✅ All setup complete! Ready to process videos.
1. Configure your processing options below:
2. Add video files to the 'Input_Videos' folder
3. Run demo_processing() to start processing


In [25]:
demo_processing()

🎯 Demo: Processing videos with current configuration...

📋 Current Configuration:
   ❌ skeleton: False
   ❌ skeleton_face_only: False
   ❌ whitebackground: False
   ❌ maskingbody: False
   ❌ maskingface: False
   ✅ blurringbody: True
   ❌ blurringface: False
   🔧 blurringfactor: 1.0
   ❌ add_finger_traces: False
   🔧 trace_length_seconds: 1.9999999999999998
   🔧 trace_color_left: (0, 255, 0)
   🔧 trace_color_right: (0, 0, 255)

📹 Found 1 video(s) to process:
   🎬 video2.mp4

🚀 Processing: video2.mp4
📹 Video Information:
   📏 Dimensions: 1280 x 720
   🎬 Frame rate: 50.0 FPS
   ⏱️ Duration: 21.4 seconds
   🎞️ Total frames: 1069


Processing frames:   0%|          | 0/1069 [00:00<?, ?it/s]

💾 Saved CSV files for video2
   📊 Body data: 1069 frames, 133 columns
   ✋ Hand data: 1069 frames, 127 columns
   😊 Face data: 1069 frames, 1435 columns
✅ Successfully processed 1069 frames
⏱️ Video duration: 21.4 seconds
📁 Output saved to: Output_Videos/video2.mp4

🎉 Processing complete! Check your Output_Videos and Output_TimeSeries folders.


# SECTION 7: EDUCATIONAL EXERCISES

In [10]:
def create_learning_exercises():
    """
    Create structured learning exercises for students
    """
    exercises = {
        "Exercise 1: Basic Understanding": {
            "description": "Explore MediaPipe's tracking capabilities",
            "tasks": [
                "1. Count the total number of trackable points",
                "2. Identify which body parts have visibility scores",
                "3. Explain the coordinate system (X, Y, Z ranges)"
            ],
            "code": """
# Your code here to explore the anatomy object
print("Body landmarks:", len(anatomy.body_landmarks))
print("Hand landmarks:", len(anatomy.hand_landmarks))
print("Face landmarks:", len(anatomy.face_landmarks))

# Try accessing specific landmark names
print("First 5 body landmarks:", anatomy.body_landmarks[:5])
            """
        },

        "Exercise 2: Configuration Exploration": {
            "description": "Experiment with different privacy settings",
            "tasks": [
                "1. Try 3 different blurring intensities",
                "2. Compare masking vs blurring effects",
                "3. Test skeleton-only mode"
            ],
            "code": """
# Your code here to test different configurations
config1 = {'blurringbody': True, 'blurringfactor': 0.3}
config2 = {'blurringbody': True, 'blurringfactor': 0.7}
config3 = {'maskingbody': True, 'skeleton': True}

# Process the same video with each config and compare
            """
        },

        "Exercise 3: Data Analysis": {
            "description": "Analyze movement patterns from CSV output",
            "tasks": [
                "1. Calculate average hand movement velocity",
                "2. Find frames with missing pose data",
                "3. Plot head movement trajectory"
            ],
            "code": """
# Your code here for data analysis
import pandas as pd
import matplotlib.pyplot as plt

# Load your processed data
# body_df = pd.read_csv('Output_TimeSeries/your_video_body.csv')

# Calculate velocities, find missing data, create plots
            """
        }
    }

    return exercises

In [11]:
print("\n📚 Learning Exercises Available:")
exercises = create_learning_exercises()
for title, content in exercises.items():
    print(f"\n{title}:")
    print(f"  📝 {content['description']}")
    for task in content['tasks']:
        print(f"     {task}")


📚 Learning Exercises Available:

Exercise 1: Basic Understanding:
  📝 Explore MediaPipe's tracking capabilities
     1. Count the total number of trackable points
     2. Identify which body parts have visibility scores
     3. Explain the coordinate system (X, Y, Z ranges)

Exercise 2: Configuration Exploration:
  📝 Experiment with different privacy settings
     1. Try 3 different blurring intensities
     2. Compare masking vs blurring effects
     3. Test skeleton-only mode

Exercise 3: Data Analysis:
  📝 Analyze movement patterns from CSV output
     1. Calculate average hand movement velocity
     2. Find frames with missing pose data
     3. Plot head movement trajectory


# SECTION 8: INTERACTIVE DASHBOARD

In [12]:
def create_comparison_demo():
    """
    Create a side-by-side comparison of different processing options
    """
    print("🔄 Creating comparison demo...")

    # Define different configurations to compare
    configs = {
        'Original': {
            'skeleton': False, 'skeleton_face_only': False, 'whitebackground': False,
            'maskingbody': False, 'maskingface': False, 'blurringbody': False,
            'blurringface': False, 'blurringfactor': 1.0, 'add_finger_traces': False,
            'trace_length_seconds': 2.0, 'trace_color_left': (0, 255, 0), 'trace_color_right': (0, 0, 255)
        },
        'Skeleton Only': {
            'skeleton': True, 'skeleton_face_only': False, 'whitebackground': True,
            'maskingbody': False, 'maskingface': False, 'blurringbody': False,
            'blurringface': False, 'blurringfactor': 1.0, 'add_finger_traces': False,
            'trace_length_seconds': 2.0, 'trace_color_left': (0, 255, 0), 'trace_color_right': (0, 0, 255)
        },
        'Body Blur + Skeleton': {
            'skeleton': True, 'skeleton_face_only': False, 'whitebackground': False,
            'maskingbody': False, 'maskingface': False, 'blurringbody': True,
            'blurringface': False, 'blurringfactor': 1.0, 'add_finger_traces': True,
            'trace_length_seconds': 2.0, 'trace_color_left': (0, 255, 0), 'trace_color_right': (0, 0, 255)
        },
        'Face Privacy + Traces': {
            'skeleton': True, 'skeleton_face_only': False, 'whitebackground': False,
            'maskingbody': False, 'maskingface': True, 'blurringbody': False,
            'blurringface': False, 'blurringfactor': 1.0, 'add_finger_traces': True,
            'trace_length_seconds': 2.0, 'trace_color_left': (0, 255, 0), 'trace_color_right': (0, 0, 255)
        }
    }

    # Check for input videos
    input_videos = [f for f in os.listdir('Input_Videos')
                   if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    if not input_videos:
        print("❌ No videos found for comparison demo in Input_Videos Folder")
        return

    video_file = input_videos[0]  # Use first available video
    print(f"🎬 Using video: {video_file}")

    # Process with each configuration
    comparison_results = {}

    for config_name, config in configs.items():
        print(f"\n🔄 Processing with {config_name} settings...")

        processor = MediaPipeProcessor(config)
        output_path = f"Output_Videos/{config_name}_{video_file}"

        try:
            results = processor.process_single_video(
                f"Input_Videos/{video_file}",
                output_path,
                save_csv=False
            )
            comparison_results[config_name] = {
                'success': True,
                'output_path': output_path,
                'frames': results['frames_processed']
            }
            print(f"✅ {config_name}: {results['frames_processed']} frames processed")

        except Exception as e:
            print(f"❌ {config_name}: Error - {str(e)}")
            comparison_results[config_name] = {
                'success': False,
                'error': str(e)
            }

    print(f"\n🎉 Comparison demo complete!")
    print(f"📁 Check Output_Videos/ folder for results")

    return comparison_results

def create_comparison_grid(video_pattern=None, output_name="comparison", max_duration=10, fps=10):
    """
    Create a side-by-side comparison video/GIF from processed videos

    Args:
        video_pattern (str): Pattern to match videos (e.g., "ted_kid") or None for all
        output_name (str): Name for output files
        max_duration (float): Maximum duration in seconds
        fps (int): Frame rate for output
    """
    print("🎬 Creating comparison video/GIF...")

    # Find all processed videos
    output_videos = get_video_list('Output_Videos')

    if video_pattern:
        output_videos = [f for f in output_videos if video_pattern.lower() in f.lower()]

    if len(output_videos) < 2:
        print(f"❌ Need at least 2 videos for comparison. Found: {len(output_videos)}")
        print(f"   Available videos: {output_videos}")
        return

    print(f"📹 Found {len(output_videos)} videos for comparison:")
    for i, video in enumerate(output_videos, 1):
        print(f"   {i}. {video}")

    # Sort videos for consistent ordering
    output_videos.sort()

    # Read all videos and get properties
    video_caps = []
    min_frames = float('inf')
    target_width, target_height = 320, 240  # Smaller size for grid

    for video_file in output_videos:
        cap = cv2.VideoCapture(f'Output_Videos/{video_file}')
        
        if cap.isOpened():
            video_fps, frame_count, _, _, _ = get_video_specs(f'Output_Videos/{video_file}')
            max_frames_allowed = int(max_duration * video_fps)
            actual_frames = min(frame_count, max_frames_allowed)
            min_frames = min(min_frames, actual_frames)
            video_caps.append((cap, video_file))
        else:
            print(f"⚠️ Could not open {video_file}")

    if not video_caps:
        print("❌ Could not open any videos")
        return

    # Calculate grid layout
    n_videos = len(video_caps)
    grid_cols = int(np.ceil(np.sqrt(n_videos)))
    grid_rows = int(np.ceil(n_videos / grid_cols))

    grid_width = target_width * grid_cols
    grid_height = target_height * grid_rows

    print(f"📐 Creating {grid_rows}x{grid_cols} grid ({grid_width}x{grid_height})")
    print(f"⏱️ Processing {min_frames} frames at {fps} FPS")

    # Setup video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    comparison_video = cv2.VideoWriter(
        f'Output_Videos/{output_name}_comparison.mp4',
        fourcc, fps, (grid_width, grid_height)
    )

    # Store frames for GIF creation
    gif_frames = []

    # Process frames
    pbar = tqdm(total=min_frames, desc="Creating comparison")

    for frame_idx in range(min_frames):
        # Create empty grid
        grid_frame = np.zeros((grid_height, grid_width, 3), dtype=np.uint8)

        # Read frame from each video
        for i, (cap, video_file) in enumerate(video_caps):
            ret, frame = cap.read()
            if ret:
                # Resize frame to fit grid
                resized_frame = cv2.resize(frame, (target_width, target_height))

                # Calculate position in grid
                row = i // grid_cols
                col = i % grid_cols

                y_start = row * target_height
                y_end = y_start + target_height
                x_start = col * target_width
                x_end = x_start + target_width

                # Place frame in grid
                grid_frame[y_start:y_end, x_start:x_end] = resized_frame

                # Add label
                label = video_file.replace('.mp4', '').replace('_', ' ')
                if len(label) > 20:
                    label = label[:17] + "..."

                cv2.putText(grid_frame, label,
                           (x_start + 5, y_start + 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

        # Save frame to video
        comparison_video.write(grid_frame)

        # Save every nth frame for GIF (to reduce size)
        if frame_idx % max(1, min_frames // 50) == 0:  # Max 50 frames for GIF
            # Convert BGR to RGB for GIF
            rgb_frame = cv2.cvtColor(grid_frame, cv2.COLOR_BGR2RGB)
            gif_frames.append(rgb_frame)

        pbar.update(1)

    # Cleanup
    pbar.close()
    comparison_video.release()
    for cap, _ in video_caps:
        cap.release()

    print(f"✅ Comparison video saved: Output_Videos/{output_name}_comparison.mp4")

    # Create GIF
    try:
        from PIL import Image
        print("🎞️ Creating GIF...")

        gif_images = [Image.fromarray(frame) for frame in gif_frames]

        gif_path = f'Output_Videos/{output_name}_comparison.gif'
        gif_images[0].save(
            gif_path,
            save_all=True,
            append_images=gif_images[1:],
            duration=int(1000/fps * max(1, min_frames // len(gif_frames))),
            loop=0
        )
        print(f"✅ GIF saved: {gif_path}")

        # Display GIF info
        gif_size = os.path.getsize(gif_path) / 1024 / 1024  # MB
        print(f"📊 GIF info: {len(gif_frames)} frames, {gif_size:.1f} MB")

    except ImportError:
        print("⚠️ PIL not available - GIF creation skipped")
        print("   Install with: pip install Pillow")
    except Exception as e:
        print(f"⚠️ GIF creation failed: {str(e)}")

    return f'Output_Videos/{output_name}_comparison.mp4'

def create_before_after_comparison(original_video, processed_videos=None):
    """
    Create a before/after comparison with the original video

    Args:
        original_video (str): Path to original video in Input_Videos/
        processed_videos (list): List of processed video names, or None for all
    """
    print("🔄 Creating before/after comparison...")

    if not os.path.exists(f'Input_Videos/{original_video}'):
        print(f"❌ Original video not found: Input_Videos/{original_video}")
        return

    # Find processed versions
    video_name = os.path.splitext(original_video)[0]
    if processed_videos is None:
        processed_videos = [f for f in os.listdir('Output_Videos')
                          if video_name.lower() in f.lower() and f.endswith('.mp4')]

    if not processed_videos:
        print(f"❌ No processed videos found for {video_name}")
        return

    # Add original video to comparison
    all_videos = [f'../Input_Videos/{original_video}'] + [f'Output_Videos/{v}' for v in processed_videos]
    labels = ['Original'] + [v.replace('.mp4', '').replace(f'{video_name}_', '') for v in processed_videos]

    print(f"📹 Comparing: {len(all_videos)} versions")
    for i, (video, label) in enumerate(zip(all_videos, labels)):
        print(f"   {i+1}. {label}")

    # Create side-by-side comparison
    caps = []
    for video_path in all_videos:
        cap = cv2.VideoCapture(video_path)
        if cap.isOpened():
            caps.append(cap)
        else:
            print(f"⚠️ Could not open {video_path}")

    if len(caps) < 2:
        print("❌ Need at least 2 videos for comparison")
        return

    # Get video properties
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_count = min([int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) for cap in caps])

    # Create grid layout
    n_videos = len(caps)
    cols = min(3, n_videos)  # Max 3 columns
    rows = int(np.ceil(n_videos / cols))

    frame_width = 300
    frame_height = 200
    grid_width = frame_width * cols
    grid_height = frame_height * rows

    # Setup output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = f'Output_Videos/{video_name}_before_after.mp4'
    out = cv2.VideoWriter(output_path, fourcc, fps, (grid_width, grid_height))

    print(f"🎬 Creating {rows}x{cols} comparison video...")

    pbar = tqdm(total=min(frame_count, int(10 * fps)), desc="Processing frames")

    for frame_idx in range(min(frame_count, int(10 * fps))):  # Max 10 seconds
        grid_frame = np.zeros((grid_height, grid_width, 3), dtype=np.uint8)

        for i, (cap, label) in enumerate(zip(caps, labels)):
            ret, frame = cap.read()
            if ret:
                # Resize and place in grid
                resized = cv2.resize(frame, (frame_width, frame_height))

                row = i // cols
                col = i % cols
                y_start = row * frame_height
                x_start = col * frame_width

                grid_frame[y_start:y_start+frame_height, x_start:x_start+frame_width] = resized

                # Add label
                cv2.putText(grid_frame, label,
                           (x_start + 10, y_start + 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        out.write(grid_frame)
        pbar.update(1)

    # Cleanup
    pbar.close()
    out.release()
    for cap in caps:
        cap.release()

    print(f"✅ Before/after comparison saved: {output_path}")
    return output_path

def auto_create_comparisons():
    """
    Automatically create comparison videos for all processed content
    """
    print("🎯 Auto-creating all comparison videos...")

    # Find unique video names
    input_videos = get_video_list('Input_Videos')
    output_videos = get_video_list('Output_Videos')

    if not output_videos:
        print("❌ No processed videos found in Output_Videos/")
        return

    # Create overall comparison of all processed videos
    print("\n1️⃣ Creating overall comparison grid...")
    create_comparison_grid(output_name="all_effects")

    # Create before/after for each original video
    for original_video in input_videos:
        video_name = os.path.splitext(original_video)[0]
        matching_processed = [f for f in output_videos if video_name.lower() in f.lower()]

        if matching_processed:
            print(f"\n2️⃣ Creating before/after for {original_video}...")
            create_before_after_comparison(original_video)

    print("\n🎉 All comparison videos created!")
    print("📁 Check Output_Videos/ folder for:")
    print("   🎬 all_effects_comparison.mp4 - Grid of all effects")
    print("   🎞️ all_effects_comparison.gif - Animated GIF")
    print("   🔄 *_before_after.mp4 - Before/after comparisons")

In [50]:
import uuid
class CreateProcessingDashboard():
    def __init__(self):
        self.output_area = widgets.Output()
        self.video_selector, self.process_button, self.analyze_button, self.upload_button, self.before_after_button, self.all_possible_outputs_button, self.compare_processed_button, self.all_input_output_button = self.create_buttons() 
        
        self.file_section = widgets.VBox([
        widgets.HTML("<h3>📁 File Management</h3>"),
        self.video_selector,
        widgets.HBox([self.upload_button, self.process_button])
        ])

        self.analysis_section = widgets.VBox([
            widgets.HTML("<h3>📊 Analysis Tools</h3>"),
            widgets.HBox([self.analyze_button])
        ])

        self.grid_section = widgets.VBox([
            widgets.HTML("<h3>📊 Compare different Videos in a Grid</h3>"),
            widgets.HBox([self.before_after_button, self.all_possible_outputs_button, self.compare_processed_button, self.all_input_output_button])
        ])

        self.dashboard = widgets.VBox([
            widgets.HTML("<h2>🎛️ MediaPipe Processing Dashboard</h2>"),
            self.file_section,
            self.analysis_section,
            self.grid_section,
            self.output_area
        ])

    def upload_handler(self, b):
        unique_id = uuid.uuid4()

        self.output_area.clear_output()
        with self.output_area:
            clear_output(wait=True)
            print("Upload button clicked", unique_id)
            print(f"id(self.output_area): {id(self.output_area)}")
            print(f"widget_repr: {repr(self.output_area)}")

    def update_video_list(self):
        videos = get_video_list('Input_Videos')
        return videos if videos else ['No videos found in Input_Videos Folder']

    def create_custom_button(self, description, button_style, on_click_handler):
        button = widgets.Button(
            description=description,
            button_style=button_style,
            layout=widgets.Layout(width='200px')
        )
        if not hasattr(button, '_handler_attached'):
            button.on_click(on_click_handler)
            button._handler_attached = True

        return button

    def create_buttons(self):
        video_selector = widgets.Dropdown(
        options= self.update_video_list(),
        description='Select Video:',
        style={'description_width': 'initial'},
        )

        process_button = self.create_custom_button(description='🚀 Process Video', button_style="success", on_click_handler=self.upload_handler)
        analyze_button = self.create_custom_button(description='📊 Analyze Data', button_style="info", on_click_handler=self.on_analyze_click)
        upload_button = self.create_custom_button(description='📤 Upload Video', button_style="primary", on_click_handler=self.on_upload_click)
        before_after_button = self.create_custom_button(description='Before/After', button_style="primary", on_click_handler=self.create_before_after_comparison_click)
        all_possible_outputs_button = self.create_custom_button(description='All Processsing Options', button_style="primary", on_click_handler=self.create_comparison_demo_click)
        compare_processed_button = self.create_custom_button(description='All Processed', button_style="primary", on_click_handler=self.create_comparison_grid_click)
        all_input_output_button = self.create_custom_button(description='All Input and Processed', button_style="primary", on_click_handler=self.auto_create_comparisons_click)

        return video_selector, process_button, analyze_button, upload_button, before_after_button, all_possible_outputs_button, compare_processed_button, all_input_output_button
    
    def on_process_click(self, b): # b is not used but is required by the library
        with self.output_area:
            clear_output(wait=True)
            selected_video = self.video_selector.value
            if selected_video and selected_video != 'No videos found in Input_Videos Folder':
                current_config = config_panel.get_config()
                processor = MediaPipeProcessor(current_config)

                input_path = f"Input_Videos/{selected_video}"
                output_path = f"Output_Videos/processed_{selected_video}"

                print(f"🚀 Processing {selected_video}...")
                try:
                    results = processor.process_single_video(input_path, output_path, save_csv=True)
                    print(f"✅ Success! Processed {results['frames_processed']} frames")

                    # Update video list in case new files were added
                    self.video_selector.options = self.update_video_list()

                except Exception as e:
                    print(f"❌ Error: {str(e)}")
            else:
                print("❌ Please select a valid video file")

    def create_before_after_comparison_click(self, b): # b is not used but is required by the library
        with self.output_area:
            clear_output(wait=True)
            selected_video = self.video_selector.value
            if selected_video and selected_video != 'No videos found in Input_Videos Folder':
                create_before_after_comparison(selected_video)
            else:
                print("❌ Please select a valid video file you want to compare")

    def create_comparison_demo_click(self, b): # b is not used but is required by the library
        with self.output_area:
            clear_output(wait=True)
            create_comparison_demo()
           
    def create_comparison_grid_click(self, b): # b is not used but is required by the library
        with self.output_area:
            clear_output(wait=True)
            create_comparison_grid()
            
    def auto_create_comparisons_click(self, b): # b is not used but is required by the library
        with self.output_area:
            clear_output(wait=True)
            auto_create_comparisons()
            
    def on_analyze_click(self, b): # b is not used but is required by the library
        self.output_area.clear_output()
        with self.output_area:
            selected_video = self.video_selector.value
            if selected_video and selected_video != 'No videos found in Input_Videos Folder':
                video_name = os.path.splitext(selected_video)[0]
                analyze_csv_output(video_name)
            else:
                print("❌ Please select a valid video file")

    def on_upload_click(self, b): # b is not used but is required by the library
        with self.output_area:
            clear_output(wait=True)
            uploaded_files = upload_sample_video()
            if uploaded_files:
                # Update video selector with new files
                self.video_selector.options = self.update_video_list()
                print(f"✅ Upload complete! Select from dropdown above.")    

In [58]:
dashboard = CreateProcessingDashboard()

In [59]:
config_panel.show_interface()

In [60]:
display(dashboard.dashboard) # why is it processing multiple times)